<a href="https://colab.research.google.com/github/ApoorvaNagarajan/Image-Filter-Kernels/blob/master/assignment4/assignment4_iteration1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import Libraries and modules**

**Idea behind the network**

- This is a vanila network
- Uses the concepts of
  - 3x3 kernels for convolution : optimal size
  - gradually increase the number of channels
  - After a certain receptive field is reached, add transition block
  - Add 1x1 as a part of the transition block : will help us reduce the number of parameters while not losing any information
  - Use as many layers as required to reach output size of 7x7 after which we can directly compute the score for each of the classes
  - Don't use Relu as the activation at the last layer. This will disregard the negative weights of classes at the final layer which may affect the accuracy
  - Keep track of validation accuracy of 1st two epochs as an initial indicator of the performance of the network

**Observations : Filled after running the network below. (Have put it in the top so that this is not missed out)**

- Parameters : 195,352
- Epochs : 10
- val_acc : 0.9921
- Time taken for each epoch : 240us
- val_acc of first two epochs: 0.9882, 0.9913
- Number of parameters are very high compared to the target, in the next iteration, I will try to reduce the parameters while trying to retain the accuracy

In [1]:
# Import all the required python libraries
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten, Add
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils

from keras.datasets import mnist

Using TensorFlow backend.


### Load pre-shuffled MNIST data into train and test sets

In [2]:
# downloads the MNIST dataset and splits it into test and train.
(X_train, y_train), (X_test, y_test) = mnist.load_data()


# X_train and X_test are reshaped from 3 dimensions to 4 dimensions
# 1st dim : num images
# 2nd and 3rd dim : Width and height of eaach of the images
# 4th dim : number of channels in each of the image. We set this to 1 as the
#           our dataset has only gray images
X_train = X_train.reshape(X_train.shape[0], 28, 28,1)
X_test = X_test.reshape(X_test.shape[0], 28, 28,1)


# Normalizing the image
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255


# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

11493376/11490434 [==============================] - 0s 0us/step


In [3]:
from keras.layers import Activation
model = Sequential()


model.add(Convolution2D(32, 3, 3, activation='relu', input_shape=(28,28,1))) # receptive field: 3x3, output dim: 26x26x32

model.add(Convolution2D(64, 3, 3, activation='relu')) # receptive field: 5x5, output dim: 24x24x64
model.add(Convolution2D(128, 3, 3, activation='relu')) # receptive field: 7x7, output dim: 22x22x128

model.add(MaxPooling2D(pool_size=(2, 2))) # receptive field: 14x14, output dim: 11x11x128
model.add(Convolution2D(32, 1, 1, activation='relu')) # receptive field: 14x14, output dim: 11x11x32

model.add(Convolution2D(64, 3, 3, activation='relu')) # receptive field: 16x16, output dim: 9x9x64
model.add(Convolution2D(128, 3, 3, activation='relu')) # receptive field: 18x18, output dim: 7x7x128

model.add(Convolution2D(10, 1, activation='relu')) # receptive field: 18x18, output dim: 7x7x10
model.add(Convolution2D(10, 7)) # receptive field: 25x25, output dim: 1x1x10
model.add(Flatten())
model.add(Activation('softmax'))


model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 24, 24, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 22, 22, 128)       73856     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 11, 11, 128)       0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 11, 11, 32)        4128      
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 9, 9, 64)          18496     
_________________________________________________________________
conv

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (3, 3), activation="relu", input_shape=(28, 28, 1...)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, (3, 3), activation="relu")`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, (1, 1), activation="relu")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), activation="relu")`
  del sys.path[0]
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher

In [0]:
model.compile(loss='categorical_crossentropy',
             optimizer='adam',
             metrics=['accuracy'])

In [6]:
model.fit(X_train, Y_train, batch_size=32, nb_epoch=10, validation_data=(X_test, Y_test), verbose=1)

Train on 60000 samples, validate on 10000 samples
Epoch 1/10
  608/60000 [..............................] - ETA: 18s - loss: 0.0243 - acc: 0.9918

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  """Entry point for launching an IPython kernel.


60000/60000 [==============================] - 15s 244us/step - loss: 0.0354 - acc: 0.9889 - val_loss: 0.0369 - val_acc: 0.9882
Epoch 2/10
60000/60000 [==============================] - 15s 243us/step - loss: 0.0280 - acc: 0.9913 - val_loss: 0.0292 - val_acc: 0.9913
Epoch 3/10
60000/60000 [==============================] - 15s 244us/step - loss: 0.0237 - acc: 0.9924 - val_loss: 0.0318 - val_acc: 0.9909
Epoch 4/10
60000/60000 [==============================] - 15s 253us/step - loss: 0.0197 - acc: 0.9939 - val_loss: 0.0424 - val_acc: 0.9871
Epoch 5/10
60000/60000 [==============================] - 15s 243us/step - loss: 0.0176 - acc: 0.9943 - val_loss: 0.0300 - val_acc: 0.9919
Epoch 6/10
60000/60000 [==============================] - 15s 242us/step - loss: 0.0144 - acc: 0.9953 - val_loss: 0.0370 - val_acc: 0.9912
Epoch 7/10
60000/60000 [==============================] - 15s 243us/step - loss: 0.0128 - acc: 0.9956 - val_loss: 0.0415 - val_acc: 0.9897
Epoch 8/10
60000/60000 [==============

In [7]:
score = model.evaluate(X_test, Y_test, verbose=0)
print(score)

[0.03546657307177038, 0.9921]
